In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-csv/Sentiment_Analysis.csv


In [2]:
df=pd.read_csv("/kaggle/input/sentiment-analysis-csv/Sentiment_Analysis.csv")

In [3]:
df

,sentiment,text
0,1,"And here is the rap song ""African Warrior Quee..."
1,1,"We asked chatGPT ,\n\nHow to become a successf..."
2,0,When I finally manage to convince myself to pe...
3,0,Although a healthier alternative to your typic...
4,1,"In my latest test, I asked #ChatGPT to write m..."
...,...,...
79995,1,This movie just might make you cooooo. The fil...
79996,0,Stack Overflow temporarily bans answers from O...
79997,0,"<span class=""tiny""> Length:: 0:49 Mins<br /><b..."
79998,1,If it is true that sadomasochism is a two-side...


In [4]:
Text=df['text']

In [5]:
!python -m spacy download en_core_web_md

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 37.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import spacy

In [7]:
nlp=spacy.load("en_core_web_md")

In [8]:
def spacy_preprocess(text):
    doc = nlp(text)
    tokens = []

    for token in doc:
            tokens.append(token)

    return tokens


    
    

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    Text, df['sentiment'], test_size=0.2, random_state=42
)

In [10]:
import emoji
import re

In [11]:
def clean_text(text):
    text = str(text).lower()
    text = emoji.demojize(text)

    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"@\w+", " ", text)
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"\\n+", " ", text)      # IMPORTANT
    text = re.sub(r"#", "", text)

    text = re.sub(r"\s+", " ", text).strip()
    return text


In [12]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # enable progress_apply

In [13]:
X_train_cleaned = X_train.progress_apply(clean_text)

100%|██████████| 64000/64000 [00:21<00:00, 2990.52it/s]


In [14]:
X_test_cleaned = X_test.progress_apply(clean_text)

100%|██████████| 16000/16000 [00:05<00:00, 2951.88it/s]


In [15]:
X_train_cleaned

3822     just about everything i've ever had from near ...
35562            is george floyd a criminal? chatgpt robot
4883     im gonna be going live in a few short minutes ...
71170    it's not my job to know anybody, anybody says ...
25665    i'm not fully convinced chatgpt but thank you ...
                               ...                        
6265     studying! but thank goodness, may sideline too...
54886    this is an instant cappuccino similar to what ...
76820    oh my god! after having such a promising start...
860      this is my first review. i am not an expert on...
15795    i think 'blackadder the third' is the best one...
Name: text, Length: 64000, dtype: object

In [16]:
X_train_cleaned_new = X_train_cleaned.progress_apply(spacy_preprocess)

100%|██████████| 64000/64000 [18:15<00:00, 58.40it/s]


In [17]:
X_test_cleaned_new = X_test_cleaned.progress_apply(spacy_preprocess)

100%|██████████| 16000/16000 [04:35<00:00, 58.08it/s]


In [18]:
X_test_cleaned_new

47044    [really, bad, weather, over, there, ?, it, 's,...
44295                        [cheatgpt, chatgpt, cheatgpt]
74783    [too, early, today, and, tomorrow, and, then, ...
70975    [these, were, properly, packed, and, consequen...
46645    [now, to, unload, van, to, go, and, stock, up,...
                               ...                        
67666    [after, playing, with, chatgpt, for, the, last...
51146    [this, movie, is, one, of, the, best, ever, pr...
42494    [my, phone, broke, new, phone, tomorrow, hopef...
52517    [i, am, disappointed, to, report, that, althou...
7754     [", rock, ', n, ', roll, high, school, ", will...
Name: text, Length: 16000, dtype: object

In [19]:
df[35562:35563]


,sentiment,text
35562,0,Is George Floyd a criminal?\nchatgpt robot\nht...


In [40]:
from gensim.models import Word2Vec

sentences = [[str(t) for t in sent] for sent in X_train_cleaned_new]#here we convert all the encoded tokens into string.ie, 125->'125'

model = Word2Vec(
    sentences=sentences,
    vector_size=200,
    window=7,
    min_count=2,
    sg=1,              # skip-gram
    negative=10,
    epochs=10
)

In [41]:
import numpy as np
embedding_dim=model.vector_size

#word2vec return a model which contain A vocabulary embedding matrix
def sentence_embedding(tokens, model):#this step is outside word2vec
    vectors = [model.wv[str(t)] for t in tokens if str(t) in model.wv]

    if len(vectors)==0:
        return np.zeros(embedding_dim)
    return np.mean(vectors, axis=0)

In [42]:
X = []
y = y_train

for text in X_train_cleaned_new:                
    vec = sentence_embedding(text, model)
    X.append(vec)

X = np.array(X)

In [43]:
from sklearn.linear_model import LogisticRegression

model_clf = LogisticRegression(max_iter=1000)
model_clf.fit(X, y)


LogisticRegression(max_iter=1000)

In [44]:
X_tst=[]
for text in X_test_cleaned_new:                
    vec = sentence_embedding(text, model)
    X_tst.append(vec)

X_tst= np.array(X_tst)

In [45]:
X_tst

array([[-0.05650483,  0.02099971, -0.02804053, ...,  0.04253388,
        -0.01300899, -0.08414158],
       [ 0.0523975 ,  0.04629683,  0.09113557, ...,  0.09847898,
        -0.23669332, -0.15916617],
       [-0.00449601,  0.09237659, -0.06655562, ...,  0.08375674,
        -0.07494903, -0.04602252],
       ...,
       [-0.00102556,  0.07342487, -0.10649961, ...,  0.08887954,
        -0.18854037, -0.22345172],
       [-0.06989802,  0.04636755, -0.06552726, ...,  0.11274654,
        -0.0119145 , -0.17757054],
       [ 0.00144181,  0.02425538, -0.08620945, ...,  0.08609367,
         0.01901875, -0.09064304]])

In [46]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1-score :", f1_score(y_test, y_pred))

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Accuracy : 0.7371875
Precision: 0.7329006490264603
Recall   : 0.7398261307798917
F1-score : 0.7363471064016552

Classification Report:

              precision    recall  f1-score   support

           0       0.74      0.73      0.74      8063
           1       0.73      0.74      0.74      7937

    accuracy                           0.74     16000
   macro avg       0.74      0.74      0.74     16000
weighted avg       0.74      0.74      0.74     16000


Confusion Matrix:

[[5923 2140]
 [2065 5872]]


In [33]:
!pip install lightgbm


In [47]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

lgbm = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

lgbm.fit(X, y)

y_pred_lgbm = lgbm.predict(X_tst)

print("Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print(classification_report(y_test, y_pred_lgbm))
print(confusion_matrix(y_test, y_pred_lgbm))


[LightGBM] [Info] Number of positive: 32063, number of negative: 31937
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500984 -> initscore=0.003938
[LightGBM] [Info] Start training from score 0.003938


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.8033125
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      8063
           1       0.80      0.81      0.80      7937

    accuracy                           0.80     16000
   macro avg       0.80      0.80      0.80     16000
weighted avg       0.80      0.80      0.80     16000

[[6409 1654]
 [1493 6444]]


In [36]:
!pip install xgboost


In [48]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

xgb.fit(X, y)

y_pred_xgb = xgb.predict(X_tst)

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))


Accuracy: 0.810875
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      8063
           1       0.80      0.82      0.81      7937

    accuracy                           0.81     16000
   macro avg       0.81      0.81      0.81     16000
weighted avg       0.81      0.81      0.81     16000

[[6458 1605]
 [1421 6516]]
